In [3]:
library(brms)

Loading required package: Rcpp

Loading 'brms' package (version 2.18.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:

    ar




In [1]:
N <- 3000
M <- 2
K <- 4
J <- 3

mu <- 0
al <- matrix(c(-1, 1), ncol = 1)
Z <- matrix(rnorm(M*N, sd = .5), ncol = N)
ep_x <- matrix(rnorm(N, sd = 0.1), ncol = N)
X <- t(al) %*% Z + ep_x

U <- matrix(rnorm(K*N, sd = .2), ncol = N)
W <- sweep(U, 2, X, "+")

h_1 <- function(x) -x
h_2 <- function(x) 3*x
f <- function(x) 2*x
eta <- rnorm(N)
S <- f(X) + h_1(Z[1, ]) + h_2(Z[2, ]) + eta
S <- matrix(S, ncol = N)

beta0 <- matrix(rep(0, J*N), ncol = N)
beta1 <- matrix(rep(1, J), ncol = 1)
ep_y <- matrix(rnorm(J * N, sd = 0.1), ncol = N)
Y <- beta0 + beta1 %*% S + ep_y

data <- as.data.frame(t(rbind(Z,X,W,Y)))
names(data) <- c("Z1", "Z2", "X","W1", "W2", "W3", "W4", "Y1", "Y2", "Y3")

data$Xm <- as.numeric(NA)
data$Sm <- as.numeric(NA)

In [4]:
bf2 <- bf(Sm|mi() ~ mi(Xm) + Z1 + Z2)
bf3 <- bf(Y1 ~ 0 + mi(Sm))
bf4 <- bf(Y2 ~ 0 + mi(Sm))
bf5 <- bf(Y3 ~ 0 + mi(Sm))
bf8 <- bf(Xm|mi() ~ Z1 + Z2)
bf9 <- bf(W1 ~ 0 + mi(Xm))
bf10 <- bf(W2 ~ 0 + mi(Xm))
bf11 <- bf(W3 ~ 0 + mi(Xm))
bf12 <- bf(W4 ~ 0 + mi(Xm))

priors <- c(
    prior(normal(0, 3), class = b, coef = Z1, resp = Sm),
    prior(normal(0, 3), class = b, coef = Z2, resp = Sm),
    prior(normal(0, 3), class = b, coef = miXm, resp = Sm),
    prior(normal(0, 3), class = b, coef = miSm, resp = Y1),
    prior(normal(0, 3), class = b, coef = miSm, resp = Y2),
    prior(normal(0, 3), class = b, coef = miSm, resp = Y3),
    prior(normal(0, 3), class = b, coef = Z1, resp = Xm),
    prior(normal(0, 3), class = b, coef = Z2, resp = Xm),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W1),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W2),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W3),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W4)
)

model <- brm(bf2 + bf3 + bf4 + bf5 + bf8 + bf9 + bf10 + bf11 + bf12 +set_rescor(FALSE), 
data = data, prior = priors, iter = 4000, cores = 4)

Compiling Stan program...

Start sampling

Warning message:
“The largest R-hat is 3.18, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#tail-ess”


In [6]:
summary(model)$fixed

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”


,Estimate,Est.Error,l-95% CI,u-95% CI,Rhat,Bulk_ESS,Tail_ESS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Sm_Intercept,0.0062694433,1.367034e-02,-0.022170066,0.030537980,1.4588423,7.680683,26.87550
Xm_Intercept,-0.0002899133,2.598795e-03,-0.005465323,0.004726165,1.0006594,7329.468913,6223.83690
Sm_Z1,-0.3446591235,5.771716e-01,-0.994188493,0.764501821,1.9313472,5.540075,32.43604
Sm_Z2,0.8582383696,1.405898e+00,-1.681069442,2.190038150,2.6169242,4.720395,25.55811
Xm_Z1,-0.9990295783,5.015224e-03,-1.008899095,-0.989171564,0.9999371,7041.819057,6726.80747
Xm_Z2,1.0044693519,5.325954e-03,0.993848818,1.014776484,1.0002417,6404.955716,7029.98308
Sm_miXm,0.4383419744,7.225850e-01,-0.920518828,1.210719961,2.1185812,5.201916,30.07152
Y1_miSm,1.0097146383,1.867866e+00,-2.200228758,2.626164772,3.1867333,4.478808,11.36788
Y2_miSm,1.0085603955,1.865734e+00,-2.198164911,2.622532679,3.1869290,4.478487,11.08886


In [7]:
save(list = ls(), file = "zero_centered.RData")